# This is a notebook to explore satellite images

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
# Don't know what that is
tf.enable_eager_execution()

# Link this somehow to Google Cloud Storage (GCS) validation images bucket directory, e.g. link can be 
# found on GCS bucket directory gs://...
training_files = dirlist('data/val/')

In [ ]:
def parse_visual(data):
    '''
    This function loads a batch of valisation satellite images and parses these images examples for 
    some specific features, e.g. spectral bands (B2, B3, B4, see official documentation)
    
    Input(s): - bunch of training files satellite images (please double check)
    Outputs:  - parsed example, e.g. images in 3D matrix? -> please double check
    '''
    dataset = tf.data.TFRecordDataset(data)
    # pattern for one part file
    # dataset = tf.data.TFRecordDataset('part-r-00099')
    iterator = dataset.make_one_shot_iterator()

    features = {
        'B2': tf.FixedLenFeature([], tf.string),
        'B3': tf.FixedLenFeature([], tf.string),
        'B4': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64),
    }

    parsed_examples = [tf.parse_single_example(data, features) for data in iterator]
    return parsed_examples

# Loads a batch of satellite images (please double check) from 'data/val (see code block above).'
parsed_examples = parse_visual(training_files[0])

In [ ]:
def get_img_from_example(parsed_example, intensify=True):
    '''
    This function creates an RGB 3D array, e.g. 65x65x3. Then it loads, for some bands / features (B4, B3, B2) a batch of satellite
    images, reshapes it from the Tensorflow specific data format into a 2D array of dimension 65x65 pixels.
    Next, it does some preprocessing, e.g. divides each pixel by the maximum value of the pixels, and 
    multiplies every pixel by 255. Only if  intensity=True (please double check).
    Otherwise,, if intensify=False, it just multiplies each value in the matrix by 255.
    last, it adds a label (please double check), and returns the image 2D array, as well as the label.
    
    Input(s): - parsed batch of satellite images: Specific Tensorflow format (please double check)
    Output(s) - label: string (please double check)
              - rgbArray: 2D numpy array (please double check)
    Utils:    - None erlier defined function uses (please double check)
    '''
    rgbArray = np.zeros((65,65,3), 'uint8')
    for i, band in enumerate(['B4', 'B3', 'B2']):
        band_data = np.frombuffer(parsed_example[band].numpy(), dtype=np.uint8)
        band_data = band_data.reshape(65, 65)
        if intensify:
            band_data = band_data/np.max(band_data)*255
        else:
            band_data = band_data*255
        rgbArray[..., i] = band_data
        
    label = tf.cast(parsed_example['label'], tf.int32).numpy()
        
    return rgbArray, label

In [ ]:
'''
This snippet should be transformed into a function with keyword arguments (**kwargs).

This snippet plots satellite images
Utils:    - get_img_from_example(...)
'''

fig=plt.figure(figsize=(20, 30), dpi= 80, facecolor='w', edgecolor='k')
for i in range(1,26):
    plt.subplot(5, 5, i)
    img, label = img, label = get_img_from_example(parsed_examples[i+7])
    plt.imshow(img).axes.get_xaxis().set_visible(False)
    plt.imshow(img).axes.get_yaxis().set_visible(False)
    plt.title(str(label))

fig.show()